In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using DataFrames
#using NearestNeighbors # Can't use KDtree in julia for posterior prediction

In [2]:
Sys.free_memory()/(2^20*1024)

20.435287475585938

In [4]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [5]:
# Packages for diagnostic test #

using StatsPlots
using MCMCChains
using PrettyTables
using Dates
using StatsBase

In [7]:
using RData
data_cleaned = load("../../data/rawdata/data_cleaned_small_expanded.RData", convert = true)["data_cleaned_small"];

In [8]:
DataFrames.describe(data_cleaned)

,variable,mean,min,median,max
,Symbol,Float64,Float64,Float64,Float64
1,x,-1.04265e7,-1.11193e7,-1.03937e7,-1.00078e7
2,y,3.83786e6,3.33608e6,3.81839e6,4.44757e6
3,scaled_x,0.6928,0.0,0.725548,1.11149
4,scaled_y,0.501779,0.0,0.482309,1.11149
5,NDVI,0.240122,-0.00954541,0.192437,0.666598
6,EVI,0.150172,-0.00400802,0.122129,0.605463
7,red_reflectance,0.144424,0.0047,0.1452,0.3892
8,NIR_reflectance,0.243485,0.0116,0.2383,0.6165
9,blue_reflectance,0.0752273,0.0001,0.0755,0.2


In [9]:
countmap(data_cleaned.LC_Type4)

Dict{Float64,Int64} with 7 entries:
  7.0 => 83009
  4.0 => 30094
  2.0 => 8421
  5.0 => 27043
  8.0 => 3111
  6.0 => 296479
  1.0 => 571843

In [10]:
hold_index_set = load("../../data/rawdata/hold_index_small_expanded.RData", convert = true);

In [11]:
#@load "../../data/LMC_data/Factor_LMC_data.jld"

In [12]:
U_indx = sort(intersect(hold_index_set["hold_ind1"], hold_index_set["hold_ind2"], 
        hold_index_set["hold_ind3"],  hold_index_set["hold_ind4"],  hold_index_set["hold_ind5"],
         hold_index_set["hold_ind6"],  hold_index_set["hold_ind7"],  hold_index_set["hold_ind8"],
         hold_index_set["hold_ind9"],  hold_index_set["hold_ind10"]));
U_ind = [hold_index_set["hold_ind1"], hold_index_set["hold_ind2"], 
        hold_index_set["hold_ind3"],  hold_index_set["hold_ind4"],  hold_index_set["hold_ind5"],
         hold_index_set["hold_ind6"],  hold_index_set["hold_ind7"],  hold_index_set["hold_ind8"],
         hold_index_set["hold_ind9"],  hold_index_set["hold_ind10"]]
N = size(data_cleaned)[1];
S_indx = setdiff(1:N, U_indx);
S_ind = [sort(setdiff(1:N, U_ind[i])) for i in 1:length(U_ind)];

In [13]:
# coordinate:
coords = [data_cleaned.scaled_x[S_indx] data_cleaned.scaled_y[S_indx]];
coords_pred = [data_cleaned.scaled_x[U_indx] data_cleaned.scaled_y[U_indx]];
n = size(coords)[1]; m = 10;

In [14]:
data_cleaned = nothing; hold_index_set = nothing; GC.gc();

In [15]:
Sys.free_memory()/(2^20*1024)

20.163818359375

In [16]:
Dates.format(now(), "HH:MM:SS")  

"00:16:23"

In [17]:
# build neighbors #
m = 10;                                                       # number of nearest neighbor                       
NN = BuildNN(coords, m, 1.0, 0.0);                       # build nearest neighbor 
nnIndx_col = vcat(NN.nnIndx, 1:n);                            # the index of columns
nnIndx_row = zeros(Int64, 0);                                               
for i in 2:m
    nnIndx_row = vcat(nnIndx_row, fill(i, i-1))
end
nnIndx_row = vcat(nnIndx_row, repeat((m + 1):n, inner = m), 1:n);  # the index of rows

In [18]:
Dates.format(now(), "HH:MM:SS")  

"00:51:28"

In [19]:
# construct Atilde Dtilde #

using RCall
@rput coords
@rput coords_pred
@rput m
R"""
library("RANN")
nn_mod_ho <- nn2(coords, coords_pred, k = m)
"""
@rget nn_mod_ho
NM = size(coords_pred)[1];
MnnIndxLU = collect(1:m:(NM * m + 1));
MnnIndx = vec(nn_mod_ho[:nn_idx]');
Mnndists = vec(nn_mod_ho[:nn_dists]');

In [20]:
Dates.format(now(), "HH:MM:SS")  

"00:51:36"

In [21]:
@save "../../data/LMC_data/Factor_LMC_data_expanded.jld" U_ind S_ind U_indx S_indx N n m NN nnIndx_col nnIndx_row NM MnnIndxLU MnnIndx Mnndists coords coords_pred